# 

In [ ]:
import os
import pickle
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so
import torch

from statsmodels.tsa.stattools import acf, pacf

os.chdir("..")  # we are supposed to be in 'TIMEVIEW/experiments/analysis', so moving up to 'experiments'
from datasets import load_dataset
from baselines import YNormalizer
from timeview.model import TTS


SAVE_DIR = "."  # change if you want different location



In [ ]:
results = pd.read_csv("benchmarks/summary_expanded.csv")
results[["method", "ord_method"]] = "Original", 0
results.loc[results["p"].notna(), ["method", "ord_method"]] = "AR(1)", 1
results.loc[results["ar_num_hidden"].notna(), ["method", "ord_method"]] = "Neural", 2

results["dataset"] = results["dataset_name"].map({
    "airfoil_log": "Airfoil (R.)",
    "flchain_1000": "flchain (R.)",
    "stress-strain-lot-max-0.2": "Stress-Strain (R.)",
    "synthetic_tumor_wilkerson_1": "Tumer (S.)",
    "sine_trans_200_20": "Sine (S.)",
    "beta_900_20": "Beta (S.)",
})

results["ord_dataset"] = results["dataset_name"].map({
    "airfoil_log": 0,
    "flchain_1000": 1,
    "stress-strain-lot-max-0.2": 2,
    "synthetic_tumor_wilkerson_1": 3,
    "sine_trans_200_20": 4,
    "beta_900_20": 5,
})
results = results.sort_values(["ord_dataset", "ord_method"])
results = results[["method", "rmse", "rmse_std", "mse", "mse_std"]]
results[["rmse", "rmse_std", "mse", "mse_std"]] = results[["rmse", "rmse_std", "mse", "mse_std"]].round(3)
results

In [ ]:
def create_latex_table(df):
    df = df.values
    out = ""
    for row in df:
        for col in row:
            out += "& " + str(col) + " "
        out += "\\\\ \n"
    return out

print(create_latex_table(results))

In [ ]:
def pad_to_array(ys: list):
    max_n = max(len(y) for y in ys)

    out = np.full((len(ys), max_n), np.nan)
    for i, y in enumerate(ys):
        out[i, :len(y)] = y
    return out


In [ ]:
def plot_time_series(dataset):
    t, y = (
        pd.DataFrame(
            pad_to_array(dataset.get_X_ts_ys()[idx])
        ).reset_index(names="id")
        for idx in [1, 2]
    )
    t = pd.melt(t, "id", value_name="t")
    y = pd.melt(y, "id", value_name="y")
    toplot = pd.merge(t, y)
    return toplot, so.Plot(
        toplot,
        x="t",
        y="y",
        color="id"
    ).add(
        so.Lines(alpha=0.3),
    )


In [ ]:
def plot_acf_pacf_hist(dataset):
    values = np.array([
        [acf(y, nlags=1)[1] for y in dataset.get_X_ts_ys()[-1]],
        [pacf(y, nlags=1)[1] for y in dataset.get_X_ts_ys()[-1]],
    ]).T
    values = pd.DataFrame(values, columns=["acf", "pacf"]).reset_index(names="id")
    toplot = pd.melt(values, "id")
    print(values.mean(0))
    return toplot, so.Plot(
        toplot,
        x="value",
    ).add(
        so.Bars(), so.Hist(),
    ).facet(col="variable")


In [ ]:
all_toplot = []

for dataset_name in (
    "airfoil_log",
    "flchain_1000",
    "stress-strain-lot-max-0.2",
    "synthetic_tumor_wilkerson_1",
    "sine_trans_200_20",
    "beta_900_20",
):
    current = load_dataset(dataset_name, dataset_description_path="dataset_descriptions")
    all_toplot.append(
        plot_time_series(current)[0].assign(dataset=dataset_name)
    )


all_toplot = pd.concat(all_toplot, ignore_index=True)
all_toplot["dataset"] = all_toplot["dataset"].map({
    "airfoil_log": "Airfoil (Real)",
    "flchain_1000": "flchain (Real)",
    "stress-strain-lot-max-0.2": "Stress-Strain (Real)",
    "synthetic_tumor_wilkerson_1": "Tumer (Synthetic)",
    "sine_trans_200_20": "Sine (Synthetic)",
    "beta_900_20": "Beta (Synthetic)",
})
# all_toplot["id"] /= all_toplot.groupby("dataset")["id"].transform("max")
# so.Plot(
#     all_toplot,
#     x="t",
#     y="y",
#     color="id"
# ).add(
#     so.Lines(alpha=0.3),
# ).facet(wrap=2)
# # all_toplot

In [ ]:
pltt = so.Plot(
    all_toplot.assign(
        id=all_toplot["id"] / all_toplot.groupby("dataset")["id"].transform("max"),
    ),
    x="t",
    y="y",
    color="id",
).add(
    so.Lines(alpha=0.3),
    legend=False,
).facet(
    col="dataset", wrap=3,
).share(
    x=False, y=False, color=False,
).layout(
    size=(10, 5),
).label(x="", y="")
pltt.show()
pltt.save(f"{SAVE_DIR}/timeseries.png")
# all_toplot

In [ ]:
dataset_summary = (
    all_toplot
    .dropna()
    .groupby(["dataset", "id"])
    .agg({"variable": "nunique"})
    .assign(n=1)
    .groupby(["dataset"])
    .agg({"n": "sum", "variable": ["mean", "std", "max"]})
    .reset_index()
)

dataset_summary.columns = [f"{a}_{b}" if b else a for a, b in dataset_summary.columns]
dataset_summary["dataset"] = dataset_summary["dataset"].map({
    "Airfoil (Real)": "Airfoil (R.)",
    "flchain (Real)": "flchain (R.)",
    "Stress-Strain (Real)": "Stress-Strain (R.)",
    "Tumer (Synthetic)": "Tumer (S.)",
    "Sine (Synthetic)": "Sine (S.)",
    "Beta (Synthetic)": "Beta (S.)",
})
dataset_summary = dataset_summary.round(1)
dataset_summary

In [ ]:
all_corrs = []
all_toplot_2 = all_toplot.copy()

for cor_type, cor_func in (
    ("ACF", acf),
    ("PACF", pacf),
):
    for diff in (0, 1):
        if diff:
            all_toplot_2["y"] = (
                all_toplot_2
                .sort_values(["dataset", "id", "variable"])
                .groupby(["dataset", "id"])["y"]
                .diff(diff)
            )
        
        all_corrs.append(
            all_toplot_2
            .dropna(subset="y")
            .groupby(["dataset", "id"])["y"]
            .apply(lambda y: cor_func(y, nlags=1)[1])
            .reset_index()
            .groupby("dataset")
            .agg({"y": "mean"})
            .rename(columns={"y": f"{cor_type}, d={diff}"})
        )
all_corrs = pd.concat(all_corrs, axis=1).round(2)
all_corrs

all_corrs = pd.concat((
    all_corrs[["ACF, d=0", "ACF, d=1"]].rename(columns=lambda s: s.replace("ACF, ", "")).assign(corr="ACF"),
    all_corrs[["PACF, d=0", "PACF, d=1"]].rename(columns=lambda s: s.replace("PACF, ", "")).assign(corr="PACF"),
)).reset_index()[["dataset", "corr", "d=0", "d=1"]]

all_corrs["ord_dataset"] = all_corrs["dataset"].map({
    "Airfoil (Real)": 0,
    "flchain (Real)": 1,
    "Stress-Strain (Real)": 2,
    "Tumer (Synthetic)": 3,
    "Sine (Synthetic)": 4,
    "Beta (Synthetic)": 5,
})

all_corrs["dataset"] = all_corrs["dataset"].map({
    "Airfoil (Real)": "Airfoil (R.)",
    "flchain (Real)": "flchain (R.)",
    "Stress-Strain (Real)": "Stress-Strain (R.)",
    "Tumer (Synthetic)": "Tumer (S.)",
    "Sine (Synthetic)": "Sine (S.)",
    "Beta (Synthetic)": "Beta (S.)",
})


all_corrs = dataset_summary.merge(all_corrs).sort_values(["ord_dataset", "corr"])  # .drop(columns=["ord_dataset", "dataset"])
all_corrs = all_corrs.astype(str)
all_corrs
for idx, row in all_corrs.iterrows():
    if idx % 2:
        all_corrs.loc[idx, ["n_sum", "variable_mean", "variable_std", "variable_max"]] = "", "", "", ""
display(all_corrs)
all_corrs = all_corrs.drop(columns=["dataset", "ord_dataset"])
print(create_latex_table(all_corrs))

In [ ]:
for dataset_name in (
    "airfoil_log",
    "flchain_1000",
    # "stress-strain-lot-max-0.2",
    # "synthetic_tumor_wilkerson_1",
    # "sine_trans_200_20",
    # "beta_900_20",
):
    current = load_dataset(dataset_name, dataset_description_path="dataset_descriptions")
    print(f"{'=' * 50} {dataset_name} {'=' * 50}")
    plot_time_series(current)[1].scale(y="log").show()
    plot_acf_pacf_hist(current)[1].show()


In [ ]:
def load_model(path):
    checkpoint = torch.load(
        f"{path}/lightning_logs/version_0/checkpoints/best_val.ckpt",
        weights_only=True,
    )
    state_dict = {k.replace("model.", ""): v for k, v in checkpoint["state_dict"].items()}
    
    with open(f"{path}/config.pkl", "rb") as f:
        config = pickle.load(f)
    model = TTS(config)
    model.load_state_dict(state_dict)

    transformer_loc = path[:path.find("TTS")] + "TTS"
    x_transformer = joblib.load(f"{transformer_loc}/column_transformer.joblib")
    
    with open(f"{transformer_loc}/y_normalizer.json", 'r') as f:
        params = json.load(f)
        y_normalizer = YNormalizer()
        y_normalizer.set_params(params['y_mean'], params['y_std'])

    
    return {
        "model": model,
        "config": config,
        "x_transformer": x_transformer,
        "y_normalizer": y_normalizer,
    }


In [ ]:
from timeview.basis import BSplineBasis
from timeview.data import _pad_to_shape


def get_forward_params(dataset, model_dict):
    with torch.no_grad():
        max_n = max(len(y) for y in dataset.ys)
        
        bspline = BSplineBasis(
            model_dict["config"].n_basis,
            (0, model_dict["config"].T),
            internal_knots=model_dict["config"].internal_knots,
        )
        Phis = list(bspline.get_all_matrices(dataset.ts))
        
        Phis = torch.stack([
            torch.from_numpy(_pad_to_shape( Phi, (max_n, model_dict["config"].n_basis))).float()
            for Phi in Phis
        ], dim=0)
        
        
        x = torch.from_numpy(model_dict["x_transformer"].transform(dataset.X)).float()
        ys = model_dict["y_normalizer"].transform(dataset.ys)
        ys = torch.stack([torch.from_numpy(_pad_to_shape(y, (max_n,))).float() for y in ys], dim=0)
    
    return x, Phis, ys


In [ ]:
results = pd.read_csv("benchmarks/summary_expanded.csv")
results[["method", "ord_method"]] = "Original", 0
results.loc[results["p"].notna(), ["method", "ord_method"]] = "AR(1)", 1
results.loc[results["ar_num_hidden"].notna(), ["method", "ord_method"]] = "Neural", 2

results["dataset"] = results["dataset_name"].map({
    "airfoil_log": "Airfoil (R.)",
    "flchain_1000": "flchain (R.)",
    "stress-strain-lot-max-0.2": "Stress-Strain (R.)",
    "synthetic_tumor_wilkerson_1": "Tumer (S.)",
    "sine_trans_200_20": "Sine (S.)",
    "beta_900_20": "Beta (S.)",
})

results["ord_dataset"] = results["dataset_name"].map({
    "airfoil_log": 0,
    "flchain_1000": 1,
    "stress-strain-lot-max-0.2": 2,
    "synthetic_tumor_wilkerson_1": 3,
    "sine_trans_200_20": 4,
    "beta_900_20": 5,
})

results = results[["dataset", "dataset_name", "ord_dataset", "method", "timestamp"]]

to_plot_all = []

for _, row in results.iterrows():
    run_time = row["timestamp"]

    path = Path(f"benchmarks/{run_time}/TTS/final/logs/seed_35492826")
    benchmarks_df = pd.read_csv("benchmarks/summary.csv")
    mod = load_model(str(path))
    
    new = load_dataset(row["dataset_name"], dataset_description_path="dataset_descriptions")
    
    with torch.no_grad():
        x, phi, y = get_forward_params(new, mod)
        pred = mod["model"](x, phi, y)
        resid = (y - pred).numpy()
        resid = [r[:len(y)] for r, y in zip(resid, current.ys)]
    new.ys = resid
    
    to_plot_all.append(
        plot_time_series(new)[0].assign(
            dataset_name=row["dataset"],
            method=row["method"],
        )
    )
to_plot_all = pd.concat(to_plot_all, ignore_index=True)
to_plot_all["method"] = pd.Categorical(
    to_plot_all["method"],
    [
        "Original",
        "AR(1)",
        "Neural",
    ],
    ordered=True,
)
to_plot_all = to_plot_all.sort_values(["dataset_name", "method", "id", "t"])
to_plot_all

In [ ]:
categories = [
    "Airfoil (R.)",
    "flchain (R.)",
    "Stress-Strain (R.)",
    "Tumer (S.)",
    "Sine (S.)",
    "Beta (S.)",
]


for pattern, data_type in (
    ("(R.)", "real"), ("(S.)", "synth")
):
    toplot_now = to_plot_all[to_plot_all["dataset_name"].str.contains(pattern)].copy()
    toplot_now["id"] /= toplot_now.groupby(["dataset_name", "method"])["id"].transform("max")
    
    toplot_now["dataset_name"] = pd.Categorical(
        toplot_now["dataset_name"],
        [cat for cat in categories if pattern in cat],
        ordered=True,
    )

    pltt = so.Plot(
        toplot_now,
        x="t",
        y="y",
    ).add(
        so.Lines(alpha=0.3),
        color="id",
        legend=False,
    ).facet(
        row="dataset_name", col="method",
    ).share(
        x=False, y=True, color=False,
    ).layout(
        size=(10, 10),
    ).label(x="", y="")
    pltt.show()
    pltt.save(f"{SAVE_DIR}/residuals_{data_type}.png")
    # all_toplot

In [ ]:
categories = [
    "Airfoil (R.)",
    "flchain (R.)",
    "Stress-Strain (R.)",
    "Tumer (S.)",
    "Sine (S.)",
    "Beta (S.)",
]


for cat in categories:
    toplot_now = to_plot_all[to_plot_all["dataset_name"] == cat].copy()
    toplot_now["id"] /= toplot_now.groupby(["dataset_name", "method"])["id"].transform("max")
    
    # toplot_now["dataset_name"] = pd.Categorical(
    #     toplot_now["dataset_name"],
    #     [cat for cat in categories if pattern in cat],
    #     ordered=True,
    # )

    pltt = so.Plot(
        toplot_now,
        x="t",
        y="y",
    ).add(
        so.Lines(alpha=0.3),
        color="id",
        legend=False,
    ).facet(
        row="dataset_name", col="method",
    ).share(
        x=False, y=True, color=False,
    ).layout(
        size=(10, 3.5),
    ).label(x="", y="")
    pltt.show()
    pltt.save(f"{SAVE_DIR}/{cat}.png")
    # all_toplot

In [ ]:
for run_time in (
    "2024-10-27T05-15-31",
    "2024-10-27T05-19-44",
    "2024-10-27T05-34-15",
):

    path = Path(f"benchmarks/{run_time}/TTS/final/logs/seed_35492826")
    benchmarks_df = pd.read_csv("benchmarks/summary.csv")
    dataset_name = benchmarks_df.loc[benchmarks_df["timestamp"] == run_time, "dataset_name"].values[0]
    mod = load_model(str(path))
    
    new = load_dataset(dataset_name, dataset_description_path="dataset_descriptions")
    
    with torch.no_grad():
        x, phi, y = get_forward_params(new, mod)
        pred = mod["model"](x, phi, y)
        resid = (y - pred).numpy()
        resid = [r[:len(y)] for r, y in zip(resid, current.ys)]
    
    
    
    new.ys = resid
    print(dataset_name)
    plot_time_series(new)[1].show()
    plot_acf_pacf_hist(new)[1].show()


In [ ]:
def nu_n(phi, n):
    toeplitz = np.abs(np.arange(n)[None,:] - np.arange(n)[:,None])
    return np.power(phi, toeplitz).sum()

In [ ]:
results = []
for phi in np.linspace(0, 1, 100):
    for n in (20, 50, 100, 200):
        results.append({
            "n": n,
            "phi": phi,
            "v_orig": nu_n(phi, n) / n ** 3,
            "v_new_series": nu_n(phi, n) / n ** 2 / (n + 1),
            "v_new_time": nu_n(phi, n + 1) / n / (n + 1) ** 2,
            "ratio": n / (n + 1) * nu_n(phi, n+1) / nu_n(phi, n)
        })
results = pd.DataFrame(results)
results["n"] = pd.Categorical(results["n"])
# results[["v_orig", "v_new_series", "v_new_time", "ratio"]] = np.sqrt(results[["v_orig", "v_new_series", "v_new_time", "ratio"]])
results["ratio"] -= 1

so.Plot(
    results,
    x="phi",
    y="ratio",
    color="n",
).add(
    so.Lines()
).scale(
    y="sqrt",
).label(
    x="Auto-correlation", y="Ratio",
).layout(
    size=(8, 4),
).save(f"{SAVE_DIR}/ratio.png")
